In [1]:
import pandas as pd

df1 = pd.read_csv(r'C:\Users\theha\OneDrive\Desktop\Projects\📊 Phân tích hiệu quả chiến dịch Marketing tháng 52025\raw data\campaign_data.csv')
df2 = pd.read_csv(r'C:\Users\theha\OneDrive\Desktop\Projects\📊 Phân tích hiệu quả chiến dịch Marketing tháng 52025\raw data\conversion_data_complex.csv')
df3 = pd.read_csv(r'C:\Users\theha\OneDrive\Desktop\Projects\📊 Phân tích hiệu quả chiến dịch Marketing tháng 52025\raw data\user_region.csv')

print(df1.info())
print(df1.head())
print(df1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campaign_id    6 non-null      float64
 1   campaign_name  6 non-null      object 
 2   budget         6 non-null      float64
 3   start_date     6 non-null      object 
 4   end_date       6 non-null      object 
dtypes: float64(2), object(3)
memory usage: 452.0+ bytes
None
   campaign_id     campaign_name  budget start_date   end_date
0          1.0       Summer Sale  5000.0  4/15/2025  5/10/2025
1          2.0    New User Promo  3000.0   5/1/2025  5/31/2025
2          3.0    Mid-Year Bonus  7000.0  5/10/2025  6/15/2025
3          4.0        Flash Deal  2000.0  5/20/2025  5/25/2025
4          5.0  Member Exclusive  4000.0   4/1/2025  5/30/2025
       campaign_id       budget
count     6.000000     6.000000
mean      3.500000  4500.000000
std       1.870829  1870.828693
min       1.000000  2000

In [2]:
print(df2.info())
print(df2.head())
print(df2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   conversion_id      541 non-null    int64  
 1   campaign_id        541 non-null    int64  
 2   user_id            541 non-null    object 
 3   conversion_date    541 non-null    object 
 4   revenue_generated  494 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 21.3+ KB
None
   conversion_id  campaign_id user_id conversion_date  revenue_generated
0              1            5   userC       5/25/2025             182.59
1              2            2   userM        5/7/2025             199.59
2              3            2  userLX       5/14/2025             181.19
3              4            5   userW       5/24/2025              91.60
4              5            4   userI       5/25/2025             284.00
       conversion_id  campaign_id  revenue_generated
c

In [3]:
print(df3.info())
print(df3.head())
print(df3.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  237 non-null    object
 1   region   237 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB
None
  user_id region
0   userC   East
1   userM   East
2  userLX  North
3   userW  South
4   userI  South
       user_id region
count      237    237
unique     237      4
top      userC   East
freq         1     69


In [4]:
# Convert date format
df1['start_date'] = pd.to_datetime(df1['start_date'], errors='coerce')
df1['end_date'] = pd.to_datetime(df1['end_date'], errors='coerce')
df2['conversion_date'] = pd.to_datetime(df2['conversion_date'], errors='coerce')

df1['campaign_id'] = df1['campaign_id'].fillna(0).astype(int)

In [5]:
# Handling Missing Values
df1.dropna(subset=['campaign_id','budget'], inplace=True) # Two blank lines with no data

In [6]:
# Handling Missing Values (df2)
# Important data used for predictive modeling
from sklearn.ensemble import RandomForestRegressor

temp_df = df2.dropna(subset=['revenue_generated'])

model = RandomForestRegressor()
model.fit(temp_df[['campaign_id', 'conversion_id']], temp_df['revenue_generated'])

missing = df2[df2['revenue_generated'].isna()]
predictions = model.predict(missing[['campaign_id', 'conversion_id']])
df2.loc[df2['revenue_generated'].isna(), 'revenue_generated'] = predictions

In [7]:
# Eliminate duplicate data
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

In [8]:
# Generate a new unique ID for conversion_id
df2['conversion_id'] = range(1, len(df2) + 1)

In [9]:
# Handling of fonts and whitespace
df2['user_id'] = df2['user_id'].str.strip().str.upper()
df3['user_id'] = df3['user_id'].str.strip().str.upper()

In [10]:
# Check and handle invalid data
# Remove negative campaign_id
df1 = df1[df1['campaign_id'] >=0]

In [11]:
# Filter campaigns with end_date in May 2025
campaign_may =  df1[(df1['end_date'].dt.month == 5) & (df1['end_date'].dt.year == 2025)]

# Filter conversions with campaign_id belonging to campaigns ending in 5/2025
campaign_ids_may = campaign_may['campaign_id'].dropna().unique()
conversion_may = df2[df2['campaign_id'].isin(campaign_ids_may)]

In [12]:
# Combine into a single summary table
df_conv_camp = pd.merge(conversion_may, df1, on='campaign_id', how='left')
df_final = pd.merge(df_conv_camp, df3, on='user_id', how='left')

In [13]:
# Copy clean data and Save each table to CSV file
df1_clean = df1.copy()
df2_clean = df2.copy()
df3_clean = df3.copy()

df1_clean.to_csv("clean_campaign_data.csv", index=False)
df2_clean.to_csv("clean_conversion_data.csv", index=False)
df3_clean.to_csv("clean_user_region_data.csv", index=False)

In [14]:
df_final.to_csv("final_conversions_may2025.csv", index=False)

In [15]:
campaign_may.to_csv("campaign_ending_may2025.csv", index=False)